# 项目分析

逐级生成分析，使用不同模板。

粗描述生成（自上而下）：
1. 首先读取每个文件，针对每个文件生成粗略描述（限制输入文本长度以免避免模型超限）
2. 根据每个文件，生成文件命名空间（父文件夹）描述（大部分引用的为命名空间）
3. 之后针对每个文件中的类生成描述，利用文件引用将其他文件的粗描述引入作为信息
4. 针对每个文件中的方法生成描述，利用类的描述信息

精细描述生成（自下而上）：
1. 利用一个类中的其他方法的描述，以及类的描述，生成当前方法描述
2. 利用类中所有方法的精细描述，生成类的描述
3. 利用一个文件中所有类的描述，生成文件的描述
4. 结合引用文件描述，重新生成当前文件描述

In [1]:
import time
import os
import pickle
from typing import *

In [2]:
import torch

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
from tinydb import TinyDB,Query

In [5]:
os.environ["RWKV_CUDA_ON"] = 'cuda:1' # if '1' then use CUDA kernel for seq mode (much faster)

In [6]:
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

In [7]:
device = 'cuda:1' if torch.cuda.is_available() else '0'

In [8]:
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']

## 1. 读取项目测试数据

In [9]:
import json

In [10]:
from chardet import detect

In [11]:
project_meta = json.load(open('../data/processed/amc-by-tree-sitter.json','r'))

In [12]:
sample_key = list(project_meta.keys())[0]

In [13]:
project_meta[sample_key]

{'filename': 'CScanPicFunction.cs',
 'filepath': '../data/AMC/AlgorithmLib/CScanPicFunction.cs',
 'source_reference': ['System.Collections.Generic',
  'System.Diagnostics',
  'System.Drawing',
  'System.IO',
  'System.Linq',
  'System.Text',
  'System.Threading',
  'System.Threading.Tasks',
  'System.Windows.Forms'],
 'source_comments': [],
 'source_class': [{'class_name': 'CScanPicFunction',
   'class_comment': '',
   'class_code': '{\r\n\r\n\r\n        /// <summary>\r\n        ///  在闸门内查找第一个与波形的交叉点\r\n        /// </summary>\r\n        /// <param name="threshValue">阈值(转换为0-2^N,N是卡的位数,譬如8位卡,就是256)</param>\r\n        /// <param name="gateLengthNum">闸门长度</param>\r\n        /// <param name="gateStartNum">闸门在A信号中的起始位置</param>\r\n        /// <param name="isPositiveThresh">计算正向阈值标志,true：正向;false:负向</param>\r\n        /// <param name="ascandata">A信号数据</param>\r\n        /// <returns></returns>\r\n        public static int FindFirstPostionInFSFGate(int threshValue, int gateLengthNum, int gateS

## 2. 读取模型参数测试

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [15]:
test_prompt = '''
'<|im_start|>使用中文的csharp资深开发者 "\n 你是一个资深的软件工程师，现在需要对一个csharp软件项目进行分析，并撰写代码文档，现在就单个代码文件进行文档撰写。\n    代码文档的格式为：\n    # 代码文件整体功能描述（400字介绍）；\n    # 代码包含的类、接口、结构体等内容描述（每个方法至少使用200字进行详细介绍。）；\n    代码文件内容如下：\n    // Copyright (c) Six Labors.\n// Licensed under the Six Labors Split License.\n\nusing SixLabors.ImageSharp.PixelFormats;\n\nnamespace SixLabors.ImageSharp;\n\n/// <summary>\n/// Extension methods for <see cref="ImageFrameCollection{TPixel}"/>.\n/// </summary>\npublic static class ImageFrameCollectionExtensions\n{\n    /// <inheritdoc cref="Enumerable.AsEnumerable{TPixel}(IEnumerable{TPixel})"/>\n    public static IEnumerable<ImageFrame<TPixel>> AsEnumerable<TPixel>(this ImageFrameCollection<TPixel> source)\n        where TPixel : unmanaged, IPixel<TPixel>\n        => source;\n\n    /// <inheritdoc cref="Enumerable.Select{TPixel, TResult}(IEnumerable{TPixel}, Func{TPixel, int, TResult})"/>\n    public static IEnumerable<TResult> Select<TPixel, TResult>(this ImageFrameCollection<TPixel> source, Func<ImageFrame<TPixel>, TResult> selector)\n        where TPixel : unmanaged, IPixel<TPixel> => source.AsEnumerable().Select(selector);\n}\n\n    请生成对应的软件文档。\n     使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（400字）。<|im_end|>\n'
'''

In [16]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()
# messages=[
#     { 'role': 'user', 'content': "write a quick sort algorithm in python."}
# ]
# inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# # tokenizer.eos_token_id is the id of <|EOT|> token
# outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
# print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("../data/deepseek-coder-6.7b-instruct-AQLM/", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("../data/deepseek-coder-6.7b-instruct-AQLM/", 
                                             # device_map = 'auto',
                                             trust_remote_code=True, 
                                             torch_dtype=torch.bfloat16).cuda()

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [17]:
messages=[
    { 'role': 'user', 'content': test_prompt}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

/home/lawbda/envs/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/lawbda/envs/anaconda3/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is

# 代码文件整体功能描述
该代码文件是一个C#项目，主要用于处理图像处理。它使用了SixLabors.ImageSharp库，该库提供了处理图像的功能。

# 代码内容描述
该代码文件包含两个类，一个是ImageFrameCollectionExtensions，另一个是ImageFrame。ImageFrameCollectionExtensions类包含两个方法，一个是AsEnumerable，另一个是Select。

AsEnumerable方法是一个扩展方法，它接受一个ImageFrameCollection作为参数，并返回该集合。这个方法的功能是返回输入的集合，无需进行任何转换。

Select方法也是一个扩展方法，它接受一个ImageFrameCollection和一个函数作为参数。这个方法的功能是将输入的集合转换为一个新的集合，该集合的元素是通过应用输入的函数得到的。

# 总结
总的来说，这个C#项目是一个图像处理工具，使用了SixLabors.ImageSharp库。它提供了两个方法，一个用于处理图像集合，另一个用于处理图像集合的选择。



## 3. 项目分析提示词模板

### 3.1 自上而下分析

#### 项目文件描述

In [14]:
def gen_prompt_csfile(content:str,
                      max_length:int=4096):
    '''
    content: 文件内容
    max_length: 最大长度
    '''

    template = f"""你是一个资深的软件工程师，现在需要对一个csharp软件项目进行分析，并撰写代码文档，现在就单个代码文件进行文档撰写。
    代码文档的格式为：
    # 代码文件整体功能描述（400字介绍）；
    # 代码包含的类、接口、结构体等内容描述（每个方法至少使用200字进行详细介绍。）；
    代码文件内容如下：
    {content[:max_length]}
    请生成对应的软件文档。
    """

    return template

In [15]:
def gen_prompt_csfile_en(content:str,
                      max_length:int=4096):
    '''
    content: 文件内容
    max_length: 最大长度
    '''

    template = f"""
    You are a senior software engineer, now we'r preparing writing the documentation for a csharp project.
    Frist we must write description for single .cs file.
    
    The code file content is as follow:
    {content[:max_length]}
    
    Following is the formate of output:
        # Overall description of the source code file
        # Class, interface, struct, method etc. description in this source file.
        # Summarization of this source code file
    
    """

    return template

#### 命名空间描述

In [16]:
def gen_prompt_namespace(file_desc_map:Dict[str,str],
                      max_length:int=512):
    '''
    file_desc_map: 文件名，生成的描述文本
    max_length: 最大长度
    '''
    template = f"""你是一个资深的软件工程师，现在需要对一个 csharp 软件项目进行分析，并撰写代码文档，现在针对一个命名空间的功能进行描述。一个命名空间可以视为包含多个代码文件的文件夹，需要撰写的文档格式如下：# 命名空间整体功能描述; # 命名空间中包含的各个文件的功能; # 根据其中包含的所有文件进行总结。该命名空间包含的文件名以及对应的文件描述内容如下：
    """
    template += '\n\n'.join(f"代码文件名:{name}\n文件描述文档:{file_desc_map[name]}" for name in file_desc_map)
    template += '\n\n请生成对应的命名空间描述文档：\n'
    return template

#### 类描述

In [31]:
def gen_prompt_class(ref_desc_map:Dict[str,str],
                     class_code:str,
                      max_length:int):
    '''
    ref_desc_map: 引用文件（命名空间）名，生成的描述文本
    class_code: 代码内容
    max_length: 最大长度
    '''
    pass

#### 方法描述

In [32]:
def gen_prompt_method(class_desc:str,
                      method_code:str,
                      max_length:int):
    '''
    class_desc: 方法所在的类的描述
    method_code: 代码内容    
    max_length: 最大长度
    '''
    pass

### 3.2 自下而上分析

In [33]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""下方给出了一个Csharp代码的方法部分的片段，该方法从一个类中抽取，可能包含文档注释以及外部的引用名称。

            # 对于本次任务的指令如下:
            {instruction}
            
            # 输入的内容如下:
            {input}
            
            # Response:
            """

## 4. 调用模型分析并存储结果

使用tinyDB将结果实时保存

In [16]:
if not os.path.isdir('../data/results'):
    os.mkdir('../data/results')

#### 自上而下-文件描述

In [19]:
db = TinyDB('../data/results/amc-rough_csfile_db-dscoderv2lite.json')

In [40]:
for idx,key in enumerate(project_meta):
    node = project_meta[key]
    # 推断文件编码格式
    fpath = node['filepath']
    ftype = get_encoding_type(fpath)
    # 用新格式打卡
    content = open(fpath, 'r', encoding = ftype, errors='ignore').read()


    # 如果已经存在则直接跳过
    query = Query()
    if db.contains(query.filepath==fpath):
        print(f'{node["filepath"]} is exist.')
        continue
    
    prompt = gen_prompt_csfile(content)
    pt_llama = new_p = f"""<|im_start|>使用中文的csharp资深开发者 "\n {prompt} 使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（800字）。<|im_end|>\n"""
    
    messages=[
        { 'role': 'user', 'content': pt_llama}
    ]
    
    inputs = tokenizer.apply_chat_template(messages, 
                                           add_generation_prompt=True, 
                                           return_tensors="pt").to(model.device)
    # tokenizer.eos_token_id is the id of <|EOT|> token
    outputs = model.generate(inputs, max_new_tokens=4086, 
                             do_sample=False, 
                             top_k=50, 
                             top_p=0.95, 
                             num_return_sequences=1, 
                             eos_token_id=tokenizer.eos_token_id)
    desc = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    # 插入数据库
    db.insert({'filepath': node['filepath'], 'desc': desc})
    print(f'{idx}: {node["filepath"]} insert sucsscfull.')

../data/AMC/AlgorithmLib/CScanPicFunction.cs is exist.
../data/AMC/AlgorithmLib/CScanPicFunctionGPU - 副本 (2).cs is exist.
../data/AMC/AlgorithmLib/CScanPicFunctionGPU - 副本.cs is exist.
../data/AMC/AlgorithmLib/CScanPicFunctionGPU.cs is exist.
../data/AMC/AlgorithmLib/UltrasonicalPeakDll.cs is exist.
../data/AMC/AlgorithmLib/obj/Debug/TemporaryGeneratedFile_036C0B5B-1481-4323-8D20-8F5ADCB23D92.cs is exist.
../data/AMC/AlgorithmLib/obj/Debug/TemporaryGeneratedFile_5937a670-0e60-4077-877b-f7221da3dda1.cs is exist.
../data/AMC/AlgorithmLib/obj/Debug/TemporaryGeneratedFile_E7A71F73-0F8D-4B9B-B56E-8E70B10BC5D3.cs is exist.
../data/AMC/AlgorithmLib/obj/Release/TemporaryGeneratedFile_036C0B5B-1481-4323-8D20-8F5ADCB23D92.cs is exist.
../data/AMC/AlgorithmLib/obj/Release/TemporaryGeneratedFile_5937a670-0e60-4077-877b-f7221da3dda1.cs is exist.
../data/AMC/AlgorithmLib/obj/Release/TemporaryGeneratedFile_E7A71F73-0F8D-4B9B-B56E-8E70B10BC5D3.cs is exist.
../data/AMC/AlgorithmLib/obj/x64/Debug/Tempor

/home/lawbda/envs/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


299: ../data/AMC/Devices/MotionCard/RobotMotionCard/MatrixClass.cs insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


300: ../data/AMC/Devices/MotionCard/RobotMotionCard/RobotMotionCard - 副本_0423.cs insert sucsscfull.


...

566: ../data/AMC/XmlConfigs/obj/x64/Release/TemporaryGeneratedFile_036C0B5B-1481-4323-8D20-8F5ADCB23D92.cs insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


567: ../data/AMC/XmlConfigs/obj/x64/Release/TemporaryGeneratedFile_5937a670-0e60-4077-877b-f7221da3dda1.cs insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


568: ../data/AMC/XmlConfigs/obj/x64/Release/TemporaryGeneratedFile_E7A71F73-0F8D-4B9B-B56E-8E70B10BC5D3.cs insert sucsscfull.
569: ../data/AMC/XmlConfigs/Properties/AssemblyInfo.cs insert sucsscfull.


#### 自上而下-命名空间描述

In [48]:
all_file_desc = {'.'.join(i['filepath'].split('/')[3:]).removesuffix('.cs'):i['desc'] for i in db.all()}

In [58]:
namespace_meta = {}
for key in project_meta:
    if not key in namespace_meta:
        namespace_meta[project_meta[key]['namespace_key']] = []
    namespace_meta[project_meta[key]['namespace_key']].append(key)

In [61]:
db_namespace = TinyDB('../data/results/amc-rough_namespace_db-dscoderv2lite.json')

In [62]:
namespace_meta

{'AlgorithmLib': ['AlgorithmLib.UltrasonicalPeakDll'],
 'AlgorithmLib.obj.Debug': ['AlgorithmLib.obj.Debug.TemporaryGeneratedFile_E7A71F73-0F8D-4B9B-B56E-8E70B10BC5D3'],
 'AlgorithmLib.obj.Release': ['AlgorithmLib.obj.Release.TemporaryGeneratedFile_E7A71F73-0F8D-4B9B-B56E-8E70B10BC5D3'],
 'AlgorithmLib.obj.x64.Debug': ['AlgorithmLib.obj.x64.Debug.TemporaryGeneratedFile_E7A71F73-0F8D-4B9B-B56E-8E70B10BC5D3'],
 'AlgorithmLib.obj.x64.Release': ['AlgorithmLib.obj.x64.Release.TemporaryGeneratedFile_E7A71F73-0F8D-4B9B-B56E-8E70B10BC5D3'],
 'AlgorithmLib.Properties': ['AlgorithmLib.Properties.Settings.Designer'],
 'AMC': ['AMC.WorkpieceStatus.Designer'],
 'AMC.AutoScan': ['AMC.AutoScan.WorkPieceAuto'],
 'AMC.DM': ['AMC.DM.TrajAxisUI.Designer'],
 'AMC.LayControl': ['AMC.LayControl.UpDownAxis.Designer'],
 'AMC.MultiArea': ['AMC.MultiArea.WorkParts'],
 'AMC.MultiAreaXml': ['AMC.MultiAreaXml.SerializeWorkPartsXml'],
 'AMC.obj.Debug': ['AMC.obj.Debug.TemporaryGeneratedFile_E7A71F73-0F8D-4B9B-B56E-

In [67]:
for idx,ns_key in enumerate(namespace_meta):
    desc_map = {key:all_file_desc[key] for key in namespace_meta[ns_key]}

    ns_prompt = gen_prompt_namespace(desc_map)
    
    # 如果已经存在则直接跳过
    query_ns = Query()
    if db_namespace.contains(query_ns.namespace==ns_key):
        print(f'{ns_key} is exist.')
        continue
    
    pt_llama_ns = f"""<|im_start|>使用中文的csharp资深开发者 "\n {ns_prompt} 使用中文回答，尽可能的详尽结合文件描述来给出当前命名空间的功能描述（800字）。<|im_end|>\n"""
    
    messages=[
        { 'role': 'user', 'content': pt_llama_ns}
    ]
    
    inputs_ns = tokenizer.apply_chat_template(messages, 
                                           add_generation_prompt=True, 
                                           return_tensors="pt").to(model.device)
    # tokenizer.eos_token_id is the id of <|EOT|> token
    outputs_ns = model.generate(inputs_ns, 
                             max_new_tokens=4086, 
                             do_sample=False, 
                             top_k=50, 
                             top_p=0.95, 
                             num_return_sequences=1, 
                             eos_token_id=tokenizer.eos_token_id)
    
    desc_ns = tokenizer.decode(outputs_ns[0][len(inputs_ns[0]):], skip_special_tokens=True)

    # 插入数据库
    db_namespace.insert({'namespace': ns_key, 'desc': desc_ns})
    print(f'{idx}: {ns_key} insert sucsscfull.')

/home/lawbda/envs/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


0: AlgorithmLib insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1: AlgorithmLib.obj.Debug insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

88: XmlConfigs.obj.Debug insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


89: XmlConfigs.obj.Release insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


90: XmlConfigs.obj.x64.Release insert sucsscfull.
91: XmlConfigs.Properties insert sucsscfull.


In [69]:
all_namespace_desc = db_namespace.all()

In [70]:
all_namespace_desc

[{'namespace': 'AlgorithmLib',
  'desc': '# 命名空间整体功能描述\n该命名空间包含了一些用于处理和分析超声波数据的C#类。这些类主要用于初始化内存，释放内存，计算峰值等。\n\n# 命名空间中包含的各个文件的功能\n该命名空间包含一个名为"UltrasonicalPeakDll"的文件，该文件是一个C#类，名为"UltrasonicalPeakDll"，它包含了一些方法和结构体。这个类主要用于处理和分析超声波数据。\n\n# 根据其中包含的所有文件进行总结\n总的来说，这个命名空间包含了一些用于处理和分析超声波数据的C#类。这些类主要用于初始化内存，释放内存，计算峰值等。\n\n# 代码文件内容\n该代码文件主要包含以下内容：\n- 定义了一个名为"UltrasonicalPeakDll"的类，该类包含了一些方法和结构体。\n- 在"UltrasonicalPeakDll"类中，定义了一个名为"CudaCGate"的结构体，该结构体用于存储一些参数，如闸门是否激活，正向阈值，负向阈值，闸门长度，闸门的起始位置等。\n- 在"UltrasonicalPeakDll"类中，定义了一些方法，如InitGPUMemory, FreeGPUMemory, FreeCPUMemory, TestTransPars, CalPeakDataPerIndex, CalPeakDataInDll, AllocCPUMemory等，这些方法用于处理和分析超声波数据。\n\n# 总结\n总的来说，这个命名空间包含了一些用于处理和分析超声波数据的C#类。这些类主要用于初始化内存，释放内存，计算峰值等。\n'},
 {'namespace': 'AlgorithmLib.obj.Debug',
  'desc': '对不起，我无法生成代码文档，因为我没有具体的代码文件来分析。我是一个AI编程助手，我可以帮助你生成代码文档，但我需要你提供具体的代码文件。\n'},
 {'namespace': 'AlgorithmLib.obj.Release',
  'desc': '对不起，我无法生成代码文档，因为我没有具体的代码文件来分析。我是一个AI编程助手，我可以帮助你生成代码文档，但我需要你提供具体的代码文件。\n'},
 {'namespace': '

In [71]:
project_meta['AMC.About']

{'filename': 'About.cs',
 'filepath': '../data/AMC/AMC/About.cs',
 'source_reference': ['System.Collections.Generic',
  'System.ComponentModel',
  'System.Data',
  'System.Drawing',
  'System.Text',
  'System.Linq',
  'System.Threading.Tasks',
  'System.Windows.Forms',
  'DevExpress.XtraEditors'],
 'source_comments': [],
 'source_class': [{'class_name': 'About',
   'class_comment': '',
   'class_code': '{\r\n        public About()\r\n        {\r\n            InitializeComponent();\r\n        }\r\n        /// <summary>\r\n        /// 禁止窗体双击关闭\r\n        /// </summary>\r\n        protected override void WndProc(ref System.Windows.Forms.Message m)\r\n        {\r\n            if (m.Msg == 0x112)\r\n            {\r\n                switch ((int)m.WParam)\r\n                {\r\n                    ////禁止双击标题栏关闭窗体\r\n                    case 0xF063:\r\n                    case 0xF093:\r\n                        m.WParam = IntPtr.Zero;\r\n                        break;\r\n                }\r\

#### 自上而下-类结构描述

In [18]:
db = TinyDB('../data/results/amc-rough_csfile_db-dscoderv2lite.json')

In [19]:
all_file_desc = {'.'.join(i['filepath'].split('/')[3:]).removesuffix('.cs'):i['desc'] for i in db.all()}

In [20]:
namespace_meta = {}
for key in project_meta:
    if not key in namespace_meta:
        namespace_meta[project_meta[key]['namespace_key']] = []
    namespace_meta[project_meta[key]['namespace_key']].append(key)

In [21]:
project_meta['AlgorithmLib.CScanPicFunction'].keys()

dict_keys(['filename', 'filepath', 'source_reference', 'source_comments', 'source_class', 'namespace_key', 'relocal_reference'])

In [35]:
def gen_prompt_class(ref_desc_map:Dict[str,str],
                     class_map:Dict[str,str],
                      max_length:int=1024):
    '''
    ref_desc_map: 引用文件（命名空间）名，生成的描述文本
    class_map: 代码内容  class_name class_comment class_code
    max_length: 最大长度
    '''
    template_pref = f"""你是一个资深的软件工程师，现在需要对一个 csharp 软件项目进行分析，并撰写代码文档，现在针对一个文件中单独的类class中包含的功能以及其作用进行描述。这个类在使用时可能调用了标准库以及本地的其他文件和命名空间，现在需要对这个类进行详细分析，需要撰写的文档格式如下：# 类的整体功能描述; # 类中包含的关键成员以及方法的介绍; # 综合上面信息，总结这个类的作用范围（如：信号分析、adapter、中间件等等）。
    这个类所引入的外部库的描述如下，分为引入的命名空间名称以及其对应的描述文本：
    """

    template_pref += '\n'.join(f"引用名：{k}；描述内容：{v}" for k,v in ref_desc_map.items())
    
    
    temlplate_middle = f"类本身的代码内容如下：\n 类名：{class_map['class_name']}；类注释：{class_map['class_comment']}；类代码体：{class_map['class_code'][:max_length]}"
    
    template_surf = '\n\n请生成对应的命名空间描述文档：\n'
    
    return template_pref + temlplate_middle + template_pref
    

In [22]:
db_namespace = TinyDB('../data/results/amc-rough_namespace_db-dscoderv2lite.json')

In [23]:
namespace_meta = {}
for key in project_meta:
    if not key in namespace_meta:
        namespace_meta[project_meta[key]['namespace_key']] = []
    namespace_meta[project_meta[key]['namespace_key']].append(key)

In [24]:
namespace_desc_map = db_namespace.all()

In [25]:
namespace_desc_map = {i['namespace']:i['desc'] for i in namespace_desc_map}

In [26]:
db_class = TinyDB('../data/results/amc-rough_class_db-dscoderv2lite.json')

In [43]:
for idx,key in enumerate(project_meta):
    node = project_meta[key]
    # 推断文件编码格式
    fpath = node['filepath']
    allref_lst = node['source_reference'] + list(node['relocal_reference'])
    ref_desc_map = {k:('' if not k in namespace_desc_map else namespace_desc_map[k]) for k in allref_lst}
    class_map_lst = node['source_class']

    for node in class_map_lst:
        class_name = node['class_name']
        class_comment = node['class_comment']
        class_code = node['class_code']
        
        # 如果已经存在则直接跳过
        query = Query()
        if db.contains(query.class_name==class_name):
            print(f'{node["class_name"]} is exist.')
            continue
        
        prompt = gen_prompt_class(ref_desc_map=ref_desc_map,class_map=node)
        
        pt_llama = new_p = f"""<|im_start|>使用中文的csharp资深开发者 "\n {prompt} 使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（800字）。<|im_end|>\n"""
        
        messages=[
            { 'role': 'user', 'content': pt_llama}
        ]
        
        inputs = tokenizer.apply_chat_template(messages, 
                                               add_generation_prompt=True, 
                                               return_tensors="pt").to(model.device)
        # tokenizer.eos_token_id is the id of <|EOT|> token
        outputs = model.generate(inputs, max_new_tokens=4086, 
                                 do_sample=False, 
                                 top_k=50, 
                                 top_p=0.95, 
                                 num_return_sequences=1, 
                                 eos_token_id=tokenizer.eos_token_id)
        desc = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    
        # 插入数据库
        db_class.insert({'filepath': fpath, 'classname':class_name,'desc': desc})
        print(f'{idx}: {fpath} - {class_name} insert sucsscfull.')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


0: ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1: ../data/AMC/AlgorithmLib/CScanPicFunctionGPU - 副本 (2).cs - CScanPicFunctionGPU insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

556: ../data/AMC/XmlConfigs/ScanAxis.cs - ScanAxis insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


557: ../data/AMC/XmlConfigs/ScanSetting.cs - ScanSetting insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


558: ../data/AMC/XmlConfigs/WaveInfo.cs - WaveInfo insert sucsscfull.
559: ../data/AMC/XmlConfigs/WaveSetting.cs - WaveSetting insert sucsscfull.


In [27]:
class_desc = db_class.all()

In [28]:
class_desc

[{'filepath': '../data/AMC/AlgorithmLib/CScanPicFunction.cs',
  'classname': 'CScanPicFunction',
  'desc': '# 类的整体功能描述\n\n该类主要用于信号分析。它提供了一些方法，这些方法用于在特定的阈值下查找第一个与波形的交叉点。\n\n# 类中包含的关键成员以及方法的介绍\n\n该类包含一个静态方法，名为FindFirstPostionInFSFGate。该方法接受多个参数，包括阈值、闸门长度、闸门在A信号中的起始位置、是否为正向阈值、A信号数据、波形数据样式、偏移量、最大振幅值、alpha值和步长。\n\n# 综合上面信息，总结这个类的作用范围\n\n该类的主要作用是进行信号分析，特别是在特定的阈值下查找第一个与波形的交叉点。\n\n# 引入的命名空间名称以及其对应的描述文本\n\n该类引入了以下命名空间：\n\n- System.Collections.Generic：用于处理集合。\n- System.Diagnostics：用于调试。\n- System.Drawing：用于处理图像。\n- System.IO：用于处理输入/输出。\n- System.Linq：用于处理查询。\n- System.Text：用于处理文本。\n- System.Threading：用于处理线程。\n- System.Threading.Tasks：用于处理并发任务。\n- System.Windows.Forms：用于处理窗口和控件。\n\n# 类本身的代码内容\n\n该类的主要功能是通过FindFirstPostionInFSFGate方法在特定的阈值下查找第一个与波形的交叉点。该方法接受多个参数，包括阈值、闸门长度、闸门在A信号中的起始位置、是否为正向阈值、A信号数据、波形数据样式、偏移量、最大振幅值、alpha值和步长。\n'},
 {'filepath': '../data/AMC/AlgorithmLib/CScanPicFunctionGPU - 副本 (2).cs',
  'classname': 'CScanPicFunctionGPU',
  'desc': '# 类的整体功能描述\n\n该类主要用于处理图像数据，包括获取版本号和读取非全波数据。\n\

#### 自上而下-方法描述

In [29]:
def gen_prompt_method(class_desc:str,
                      method_code:str,
                      max_length:int=1024):
    '''
    class_desc: 方法所在的类的描述
    method_code: 代码内容    
    max_length: 最大长度
    '''
    template_pref = f"""你是一个资深的软件工程师，现在需要对一个 csharp 软件项目进行分析，并撰写代码文档，现在针对一个文件中类class中包含的method功能以及其作用进行描述。现在给定当前类的功能描述以及方法代码，现在需要对这个方法进行分析，并撰写方法的描述文档：# 方法的的参数含义以及返回值的含义; # 方法所实现的功能; # 方法是如何实现的，使用了什么算法。
    该方法对应的类的描述如下：{class_desc[:max_length]}
    
    方法代码如下:{method_code[:max_length]}
    """
    
    template_surf = '\n\n请生成对应的命名空间描述文档：\n'
    
    return template_pref + template_pref

In [30]:
class_desc_map = {}
for node in class_desc:
    key = node['filepath']
    if not key in class_desc_map:
        class_desc_map[key] = {}
    class_name = node['classname']
    class_desc_map[key][class_name] = node['desc']

In [44]:
db_method = TinyDB('../data/results/amc-rough_method_db-dscoderv2lite.json')

In [47]:
for idx,key in enumerate(project_meta):
    node = project_meta[key]
    # 推断文件编码格式
    fpath = node['filepath']
    allref_lst = node['source_reference'] + list(node['relocal_reference'])
    ref_desc_map = {k:('' if not k in namespace_desc_map else namespace_desc_map[k]) for k in allref_lst}
    class_map_lst = node['source_class']
    for class_map_node in class_map_lst:
        class_name = class_map_node['class_name']
        class_desc_str = class_desc_map[fpath][class_name]
        
        method_map_lst = class_map_node['methods']
        for method_node in method_map_lst:
            
            method_name = method_node['method_name']
            query = Query()
            if db_method.contains(query.methodname==method_name):
                print(f'{method_name} is exist.')
                continue
            
            method_comment = method_node['method_comment']
            method_code = method_node['method_code']
            method_prefix = method_node['method_prefix']

            prompt_str = gen_prompt_method(class_desc=class_desc_str,method_code=method_code)
            
            pt_llama = new_p = f"""<|im_start|>使用中文的csharp资深开发者 "\n {prompt_str} 使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（800字）。<|im_end|>\n"""
        
            messages=[
                { 'role': 'user', 'content': pt_llama}
            ]
            
            inputs = tokenizer.apply_chat_template(messages, 
                                                   add_generation_prompt=True, 
                                                   return_tensors="pt").to(model.device)
            # tokenizer.eos_token_id is the id of <|EOT|> token
            outputs = model.generate(inputs, max_new_tokens=4086, 
                                     do_sample=False, 
                                     top_k=50, 
                                     top_p=0.95, 
                                     num_return_sequences=1, 
                                     eos_token_id=tokenizer.eos_token_id)
            desc = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
        
            # 插入数据库
            db_method.insert({'filepath': fpath, 'classname':class_name,'methodname':method_name ,'desc': desc})
            print(f'{idx}: {fpath} - {class_name} - {method_name} insert sucsscfull.')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


FindFirstPostionInFSFGate is exist.
FindFirstPostionRelativeInGate is exist.
CalCouplantDepth is exist.
DataToColor is exist.
DataToColorForInt is exist.
DataToColorForInt is exist.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


0: ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - OutOldColorBar insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

548: ../data/AMC/XmlConfigs/CorrectWave.cs - CorrectWave - InitWaveSetting insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


550: ../data/AMC/XmlConfigs/DPRConfig.cs - DPRConfig - InitDPRSetting insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


556: ../data/AMC/XmlConfigs/ScanAxis.cs - ScanAxis - IndexAxisesToDictionary insert sucsscfull.
557: ../data/AMC/XmlConfigs/ScanSetting.cs - ScanSetting - DefaultEnableAxisToArray insert sucsscfull.
